<a href="https://colab.research.google.com/github/21cs076/Detection-of-Landslides/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics==8.3.72

In [ ]:
from ultralytics import YOLO
from google.colab import files

model = YOLO("yolo11s.pt")
model.train(
    data = "/content/drive/MyDrive/Landslide_Dataset/data.yaml",  # Path to your data.yaml file
    epochs=80,  # Number of epochs (adjust based on your needs)
    batch=16,   # Batch size (adjust based on GPU memory), (16), 32, 64, 128
    imgsz=640,  # Image size for training ((640 is a good choice), 736)
    project = "/content/drive/MyDrive/Landslide_Dataset",  # Directory to save training results
    name="runs",  # Name for the training run
    exist_ok=True  # Overwrite existing results if any
)
files.download("/content/drive/MyDrive/Landslide_Dataset/runs/weights/best.pt")

New https://pypi.org/project/ultralytics/8.3.75 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.72 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/drive/MyDrive/Landslide_Dataset/data.yaml, epochs=80, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Landslide_Dataset, name=runs, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None

train: Scanning /content/drive/MyDrive/Landslide_Dataset/train/labels... 4592 images, 388 backgrounds, 0 corrupt: 100%|██████████| 4592/4592 [01:43<00:00, 44.48it/s] 


train: New cache created: /content/drive/MyDrive/Landslide_Dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.4' (you have '2.0.3'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/drive/MyDrive/Landslide_Dataset/valid/labels... 575 images, 49 backgrounds, 0 corrupt: 100%|██████████| 575/575 [00:09<00:00, 61.45it/s] 


val: New cache created: /content/drive/MyDrive/Landslide_Dataset/valid/labels.cache
Plotting labels to /content/drive/MyDrive/Landslide_Dataset/runs/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to /content/drive/MyDrive/Landslide_Dataset/runs
Starting training for 80 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      13/80      4.35G      1.649      1.282      1.239         36        640: 100%|██████████| 287/287 [03:04<00:00,  1.56it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:13<00:00,  1.30it/s]

                   all        575        958      0.663      0.547      0.631      0.304



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      14/80      4.35G      1.628      1.249      1.225         37        640: 100%|██████████| 287/287 [03:05<00:00,  1.55it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:14<00:00,  1.27it/s]

                   all        575        958      0.678      0.645      0.703      0.352


In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/Landslide_Dataset/runs/weights/best.pt")
#model = YOLO("/content/drive/MyDrive/models/lbest70m.pt")

metrics = model.val(data="/content/drive/MyDrive/Landslide_Dataset/data.yaml", split='test')
print(f"mAP50: {metrics.box.map50}") # mAP refers to the Average Precision at a single IoU threshold, often 50%.
print(f"mAP75: {metrics.box.map75}") # mAP at IoU threshold of 75% (mAP75)

Ultralytics 8.3.72 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 238 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /content/Landslide-Detection-Augmentation-12/test/labels... 60 images, 21 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<00:00, 346.61it/s]

val: New cache created: /content/Landslide-Detection-Augmentation-12/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.73it/s]


                   all         60         97      0.603       0.55      0.542      0.269
               Fissure         19         30       0.53      0.533      0.476      0.248
             Landslide         30         67      0.676      0.567      0.607      0.289
Speed: 0.3ms preprocess, 18.1ms inference, 0.0ms loss, 4.0ms postprocess per image
Results saved to runs/detect/val
mAP50: 0.5415869063090599
mAP75: 0.25017783714653685
